# Anlysis of Museum Collection

## Dataset and Motivation

- Out dataset is [The Metropolitan Museum of Art Open Access]( https://github.com/metmuseum/openaccess/ ) in CSV format. The Metropolitan Museum of Art has waived all copyright and related or neighboring rights to this dataset to inspire more researchers exploring it.
- This dataset include identifying data of more than 470,000 artworks in the museum collection. It provides detailed information of each artworks, which reveal huge potential of data analyzing work.
- At the same time, the meta data is full of data cleaning problems, such as the data was put in the wrong places, missing values, missing documentation, inconsistent information, possible duplication, mixed text and numeric data. It is a really good dataset for practicing data cleaning work.

The main attributes of the dataset are:
- **Object Number/Object ID**: a unique number sequence for each object in museum.
- **Is Highlight**: a bool to show whether an object is highlighted or not.
- **Is Public domain**: a bool to show whether an object is displayed in public domain or not.
- **Department**: the department the object belongs to.
- **Object name**: how the object called.
- **Title**: detailed object name with discription.
- **Culture/Period/Dynasty/Reign**: the background of each object.
- **Portfolio**: the portfolio the object belongs to.
- **Artist Pole/Artist Prefix/Artist Display Name/Artist Display Bio/Artist Suffix/Artist Alpha Sort/Artist Nationality/Artist Begin Date/Artist End Date**: all detailed artist background.
- **Object Date/Object Begin Date/Object End Date**: the display period of each object.
- **Medium**: the materials using to made the items.
- **Dimensions**: the dimension of each object.
- **Credit Line**: the credit information of each object.
- **Classification**: the category of material each objects belongs to.

## Research Questions

We are interested in the distribution of all the collections, which could be analyzed by storing departments, collection types, credit line history and roles of artist.
        
- The objects and corresponding departments
    - Input: Department name extracted from the Department column.
    - Output: A visual check of the number of artworks in each department.

- The objects and corresponding types
    - Input: Object type extracted from the Object_Name column. This column only contains object type. The detailed discription is stored in Object_Title.
    - Output: A visual check of several top amount of artworks' types that are collected in the museum.

- The objects and corresponding credit line
    - Input: Credit year extracted from the Credit_Line column.
    - Output: A visual check of the credit line frequency in the history by time line.
- The artist roles
    - Input: Artist roled extracted from the Artist_Role column.
    - Output: A visual check of most popular roles who produced masterpieces in history. 

## Literature Reviews

An article about data cleaning for museum collection has been found on Google: [Data Cleaning with Python — MoMAs Artwork Collection]( https://www.dataquest.io/blog/data-cleaning-with-python )
    
**What does the author do**

    TBD

**What do we do**

    TBD

## Data cleaning

As we talked before, the meta data is full of data cleaning problems. We intend to explore the following problems:

+ Check if any spelling mistakes in the Repository column
+ Extract the year information in Credit_Line column and store it into an new column called "Credit_Year"
+ Roughly check if any spelling mistakes in Artist_Display_Name column, which shoud corresponds to the Artist_Alpha_Sort. Then store the judgement into an new column called "Artist_Name_Check"
+ Check the Artist_Role column, reformatting it and use the reformatted data to replace the raw data.
+ Change the datatype of the Metadata_Date columns to datetime data.

This data cleaning section is intended to clean the dataset rather than formating the dataset. So in case of losing raw information, we intend to store the cleaned data into new columns and preserve the raw data.

### Import all the module required

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from collections import Counter, defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from wordcloud import WordCloud
from math import inf
import data_clean_functions as dcf

### Import the data from the website

In [ ]:
collection_df = pd.read_csv("https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv")

In original columns, some column names have space in it, which could be difficult to call. So we modify the names of columns for indexing convenience.

In [ ]:
collection_df.columns = ['_'.join(col_name.split()) for col_name in collection_df]

### First look of the data

In [ ]:
collection_df.dtypes

We could check number of NaNs in all the columns first and try to ingnore those columns whose most values are NaN. Those columns almost filled with contents are more likely to give us general patterns of the dataset, which should be analyzed prior. These kind of columns include Object_Number, Is_Highlight, Is_Public_Domain, Object_ID, Department, Object_Name and etc.

We give a practical example of dropping all the columns whose NaN values are more than 10%. But we won't use this for practicing perpose.

In [ ]:
collection_df.isna().sum()

In [ ]:
filtered_df =  collection_df.loc[:, collection_df.isna().sum(axis = 0) < len(collection_df.index)*0.1 ]
filtered_df

### Check the amount of repository
We already know the Metropolitan Museum of Art in New York has only one repository. So we check this in case of spelling mistakes.

In [ ]:
collection_df.Repository.value_counts()

Just one row index name. This meets our expectation.

### Extract the year information in Credit_Line column

Extract the credit year from Credit_Line columns and use the year to add an new column called 'Credit_Year' with timeline data type. The credit year is always documented at the end of credit line information, which is easy to extract. The museum is founded in 1870. So we could filter the wrong numebr of years which exceed the [1870,2019] range of time and convert the datatype to timeline data.

In [ ]:
def credit_year(row):
    if pd.notna(row):
        ## find the year in string type document
        year_str = re.compile('\d{4}').findall(row.split(',')[-1])
        if year_str:
            year_int = int(year_str[0])
            ## filter years according to the found year of the museum
            if 1870 <= year_int <= 2019:
                return pd.Timestamp(str(year_int))
    return np.nan
collection_df['Credit_Year'] = collection_df.Credit_Line.apply(credit_year)

### A light check of Artist Display Name
The Artist_Display_Name column shoud correspond to the Artist_Alpha_Sort column. These two columns should keep the same content, excepting from abbreviation, alpha sorting, title ignorance.

These are really "dirty" data, which has lots of problems, like spelling mistakes, full name vs abbreviation, missing titles.

We try to compare the tokenized Artist_Display_Name and Artist_Alpha_Sort column. Compare the percentage of similarity and set the roughly convincing rate to 60% due to abbreviation and title existence. The actually spelling mistake should get more than that.

Obviously this is a really awkward way to check the names. We actually tried to several other methods which import an named detecting module:

+ from nameparser.parser import HumanName
+ from nltk.tag.stanford import NERTagger

Both of them produced unsatifying results, espetially when applied to the Artist_Alpha_Sort column. There seems no best way to do human name comparation, espetially the comparation between full name and abbreviation. It's time-consuming and ugly to write. So We choose to implement a rough check.

In [ ]:
def check_name(row):
    if pd.isna(row.Artist_Alpha_Sort):
        return True
    else:
        r1 = re.findall(r'\w+',row.Artist_Display_Name.lower())
        r2 = re.findall(r'\w+',row.Artist_Alpha_Sort.lower())
        count = 0
        for word in r1:
            if word in r2:
                count += 1
        if count/len(r1) > 0.6:
            return True
        count = 0
        for word in r2:
            if word in r1:
                count += 1
        if count/len(r2) > 0.6:
            return True
        else:
            return False
collection_df['Artist_Name_Check'] = collection_df[collection_df.Artist_Display_Name.notna()].apply(check_name,1)

### Check the Artist_Role column

We intend to check th spelling mistake of Artist_Role column and merge the Artist_Prefix column to it. This should give us a thinking of some most popular roles in history.

The data is quite well formatted already. We just need to do some reformatting work. Remove the adjectives and leave the roles only, split the multipal conbined roles and rejoin them.

This is definitely extract information by NLP. We ignore some processes as the data is just seperated words rather than sentences.

In [ ]:
## expanding contractions
contraction_patterns=[(r'can\'t', 'cannot'),
                    (r'haven\'t', 'have not'),
                    (r'(\w+)\'ll', '\g<1> will'),
                    (r'(\w+)\'re', '\g<1> are')]

class contraction_replacer(object):
    def __init__(self, contraction_patterns):        
        # store compiled regex object
        self._contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]
        
    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self._contraction_regexes:
            text = contraction_regex.sub(replaced_text,text)
        return text

def get_noun(row):
    sample_contraction_replacer = contraction_replacer(contraction_patterns)
    ## word tokenize
    if pd.isna(row):
        return row
    else:
        text = re.sub('\|',' ',row)
    words = nltk.tokenize.word_tokenize(sample_contraction_replacer.do_contraction_normalization(text))
    words = set(words)

    ## stop words removal
    stopwords = nltk.corpus.stopwords.words('english')
    words = [w for w in words if w not in stopwords]

    ## lemmatization
    wnetl = WordNetLemmatizer()

    for i in range(len(words)):
        if not wordnet.synsets(words[i]):
            nword = wnetl.lemmatize(words[i], 'n')
            if wordnet.synsets(nword):
                words[i] = nword
    return '|'.join(words)

collection_df.Artist_Role = collection_df.Artist_Role.apply(get_noun,1)

## Data analysis & Data Visualization

### Explore the general information of the data set
Check the percentage of highlighted collections from the Is_Highlight column and the percentage of public domain collections from the Is_Public_Domain columns

In [ ]:
highlight_se = collection_df.Is_Highlight.value_counts()
quantile_highlight = highlight_se[1]/(highlight_se[0] + highlight_se[1])
public_se = collection_df.Is_Public_Domain.value_counts()
quantile_public = public_se[1]/(public_se[0] + public_se[1])
print(f'The percentage of highlighted collections is {quantile_highlight * 100}%')
print(f'The percentage of public domain collections is {quantile_public * 100}%')

From the results we can find that the highlighted collections is no more than 0.40%. The no more than half of the collections are in public domain.

### Explore the possible spelling mistake in Artist_Display_Name
This is a really rough check of the possible spelling mistake in Artist_Display_Name columns. Under the pre-condition in data cleaning process, only a few spelling mistake exits, which require human check.

In [ ]:
print(collection_df.Artist_Name_Check.value_counts(dropna=False))
collection_df[['Artist_Name_Check','Artist_Display_Name','Artist_Alpha_Sort']].sample(10)

### Explore the types of collections
The Object_Name column shows the type of each object. We are interested to find the larggest amount of object types in the museum collection.

In [ ]:
top_ten_collections = collection_df.Object_Name.value_counts(dropna=False).head(10)
top_ten_collections

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = top_ten_collections.values, y = top_ten_collections.index, alpha=0.8, orient='h')
plt.title('Top Ten Collection Types in The Museum')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Collection Types', fontsize=12)
plt.show()

### Explore the department
The Department column shows the department where each object belongs to. We are interested to take a loot at the collections of each department.

In [ ]:
departments = collection_df.Department.value_counts()
departments

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = departments.values, y = departments.index, alpha=0.8, orient='h')
plt.title('Quantiles of Collections in Different Departments')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Names of Departments', fontsize=12)
plt.show()

### Explore the frequency of credit line
The Credit_Year column we build in data cleaning section shows the the year of the collection been collected. We want to look at the frequency of the credit to check if the frequency increases or decreases with time flows.

In [ ]:
freq_df = collection_df.Credit_Year.value_counts()
ax = freq_df.plot(figsize = (15,5))
freq_df.rolling(window = 30).mean().plot(ax= ax, color='green', label = 'frequnecy')

### Explore the most popular artist role

The Artist_Roles column is already been cleaned and reformated. We just need to extract them, count the frequency of each role and store it into  a dictionary. There are too many roles, a top ten display should be enough to give some insights. Also a wordnet plot is given for vivid check.

In [ ]:
def select_top_ten_roles(df):
    roles = defaultdict(int)
    for i in df[df.Artist_Role.notna()].index:
        for role in df.Artist_Role.loc[i].split('|'):
            roles[role] += 1
    x, y = [], []
    upper_bound = inf
    for _ in range(10):
        lar = -inf
        for key,val in roles.items():
            if lar < val < upper_bound:
                lar = val
                temp = key
        upper_bound = lar
        x.append(lar)
        y.append(temp)
    return x, y, roles
x, y, roles = select_top_ten_roles(collection_df)

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = x, y = y, alpha=0.8, orient='h')
plt.title('Artist Role Count')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Names of Roles', fontsize=12)
plt.show()

In [ ]:
text = ''
for key in roles:
    text += (key+' ')*roles[key]
# text = " ".join(words)
wordcloud = WordCloud().generate(text)

plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear',)
plt.axis("off")


plt.show()

## Conclusion

**Main Feature**: In this project we are focused on how to clean string data, extract informations from the data bu using regex and NLP. The natural way to analyze the cleaned str data is to count the frequency by category. We tried visualize the frequency both by category and by timeline.

**Summary**: The whole project should give others a general look of how the artworks in The Metropolitan Museum of Art distributed with different scales.

**Future work**: A further deep analysis could be implemented on the credit year timeline to check if there is any event could be connected with the peak or valley of the credit line frequency. Or we can build a network between the Metropolitan Museum of Art and other museums to see the artworks' flow pattern.